<a href="https://colab.research.google.com/github/eemmy2547/229352/blob/main/lab03_knn_using_grid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #3

In [ ]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


In [ ]:
print("X[0]:", Xtrain[0])
print("y[0]:", ytrain[0])

X[0]: From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





y[0]: 7


In [ ]:
train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Apply Tfidf ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# define transformation
tfidf = TfidfVectorizer(stop_words='english')

# fit+transform training set
Xtrain_tfidf = tfidf.fit_transform(Xtrain)

# See output
Xtrain_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 341881 stored elements and shape (3000, 61689)>

In [ ]:
#tfidf.vocabulary_

#### Exercise 1: Find post in the training set that is closest in tf-idf to the first post in the test set (`Xtest[0]`). Print the content of both posts (not the tf-idf vectors).

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

Xtest_tfidf = tfidf.transform(Xtest)

similarities = cosine_similarity(Xtest_tfidf[0], Xtrain_tfidf)

most_similar_idx = np.argmax(similarities)

print("Test Post [0]:")
print(Xtest[0])
print("\n")
print("The closest Training Post:")
print(Xtrain[most_similar_idx])

Test Post [0]:
From: v064mb9k@ubvmsd.cc.buffalo.edu (NEIL B. GANDLER)
Subject: Need info on 88-89 Bonneville
Organization: University at Buffalo
Lines: 10
News-Software: VAX/VMS VNEWS 1.41
Nntp-Posting-Host: ubvmsd.cc.buffalo.edu


 I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.

			Neil Gandler



The closest Training Post:
From: v063kcbp@ubvmsd.cc.buffalo.edu (MITCH)
Subject: Thanks! (Backing Up Masters)
Keywords: Misled, Confused, Advice, Comprendo!
Organization: University at Buffalo
Lines: 8
News-Software: VAX/VMS VNEWS 1.41
Nntp-Posting-Host: ubvmsd.cc.buffalo.edu


	Just a 

### Classify with k-Nearest Neighbor (kNN) ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Define model
nb = KNeighborsClassifier(n_neighbors=5)

# Fit the model
nb.fit(Xtrain_tfidf, ytrain)


KNeighborsClassifier()

Evaluate on the test set using [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

We will focus on the [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [ ]:
from sklearn.metrics import classification_report

# Transform the test set
Xtest_tfidf = tfidf.transform(Xtest)

# Make predictions on the test set
ypred = nb.predict(Xtest_tfidf)

# Report classification scores
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.35      0.43      0.38        21
           1       0.33      0.48      0.39        21
           2       0.46      0.62      0.52        26
           3       0.45      0.56      0.50        34
           4       0.62      0.47      0.53        34
           5       0.88      0.54      0.67        26
           6       0.35      0.55      0.43        22
           7       0.53      0.57      0.55        28
           8       0.80      0.73      0.76        33
           9       0.77      0.68      0.72        25
          10       0.67      0.74      0.70        27
          11       0.76      0.95      0.84        20
          12       0.36      0.21      0.26        24
          13       0.78      0.61      0.68        23
          14       0.84      0.75      0.79        28
          15       0.62      0.62      0.62        29
          16       0.56      0.71      0.62        21
          17       0.82    

### Combine all methods into a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words = 'english')),
                     ('nb', KNeighborsClassifier(n_neighbors=5))])

# Fit the pipeline to the training set
pipeline.fit(Xtrain, ytrain)

# Make predictions on the test set
ypred = pipeline.predict(Xtest)

# report classification scores
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.35      0.43      0.38        21
           1       0.33      0.48      0.39        21
           2       0.46      0.62      0.52        26
           3       0.45      0.56      0.50        34
           4       0.62      0.47      0.53        34
           5       0.88      0.54      0.67        26
           6       0.35      0.55      0.43        22
           7       0.53      0.57      0.55        28
           8       0.80      0.73      0.76        33
           9       0.77      0.68      0.72        25
          10       0.67      0.74      0.70        27
          11       0.76      0.95      0.84        20
          12       0.36      0.21      0.26        24
          13       0.78      0.61      0.68        23
          14       0.84      0.75      0.79        28
          15       0.62      0.62      0.62        29
          16       0.56      0.71      0.62        21
          17       0.82    

Now we will use [grid search cross-validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find model with the best hyperparameters

![5CV](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'tfidf__stop_words': ['english'],
    'nb__n_neighbors': [1, 3, 5], # Fixed typo here
}

# Define GridSearchCV
gridcv = GridSearchCV(pipeline, params, scoring='f1_macro', cv=3)

# Fit and cross-validate the model
gridcv.fit(Xtrain, ytrain)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words='english')),
                                       ('nb', KNeighborsClassifier())]),
             param_grid={'nb__n_neighbors': [1, 3, 5],
                         'tfidf__stop_words': ['english']},
             scoring='f1_macro')

In [ ]:
gridcv.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('nb', KNeighborsClassifier(n_neighbors=1))])

In [ ]:
# Make predictions on the test set
ypred = gridcv.predict(Xtest)

# Report classification scores
print(classification_report(ytest, ypred))


              precision    recall  f1-score   support

           0       0.47      0.38      0.42        21
           1       0.37      0.48      0.42        21
           2       0.50      0.58      0.54        26
           3       0.43      0.47      0.45        34
           4       0.68      0.50      0.58        34
           5       0.59      0.50      0.54        26
           6       0.38      0.41      0.39        22
           7       0.64      0.50      0.56        28
           8       0.83      0.76      0.79        33
           9       0.59      0.68      0.63        25
          10       0.70      0.59      0.64        27
          11       0.67      0.80      0.73        20
          12       0.38      0.38      0.38        24
          13       0.73      0.70      0.71        23
          14       0.64      0.75      0.69        28
          15       0.57      0.69      0.62        29
          16       0.62      0.62      0.62        21
          17       0.75    

#### Exercise 2:

1. Use grid search 5-fold cross-validation across different values of the following two kNN parameters: `n_neighbors` and `metric`  **on the training set** to find the best model.

2. For the best value of `n_neighbors` and `metric` you found above, compute the `f1_macro` score **on the test set**.
* Print the value of `n_neighbors` and `metric`.
* Print the model's `f1_macro` score.

In [ ]:
params = {
    'tfidf__stop_words': ['english'],
    'nb__n_neighbors': [1, 3, 5],
    'nb__metric': ['euclidean', 'manhattan', 'minkowski']
}

# Define GridSearchCV
gridcv = GridSearchCV(pipeline, params, scoring='f1_macro', cv=5)

# Fit on training data
gridcv.fit(Xtrain, ytrain)

# Get best model
best_model = gridcv.best_estimator_

# Predict on test data
ypred = best_model.predict(Xtest)

# Compute f1_macro score
print(classification_report(ytest, ypred))

# Print results
print("Best n_neighbors:", gridcv.best_params_['nb__n_neighbors'])
print("Best metric:", gridcv.best_params_['nb__metric'])


              precision    recall  f1-score   support

           0       0.47      0.38      0.42        21
           1       0.37      0.48      0.42        21
           2       0.50      0.58      0.54        26
           3       0.43      0.47      0.45        34
           4       0.68      0.50      0.58        34
           5       0.59      0.50      0.54        26
           6       0.38      0.41      0.39        22
           7       0.64      0.50      0.56        28
           8       0.83      0.76      0.79        33
           9       0.59      0.68      0.63        25
          10       0.70      0.59      0.64        27
          11       0.67      0.80      0.73        20
          12       0.38      0.38      0.38        24
          13       0.73      0.70      0.71        23
          14       0.64      0.75      0.69        28
          15       0.57      0.69      0.62        29
          16       0.62      0.62      0.62        21
          17       0.75    